# <center> ML Lab 2 Project

### <center> Mathew Varghese

# Reading data and imports

In [526]:
# Imports
import pandas as pd
import numpy as np
from rdkit import Chem
import time
import tensorflow as tf
import tensorflow_probability as tfp

In [527]:
# Accessing google drive
from google.colab import drive
drive.mount('/content/drive')
#loading files
df = pd.read_csv('/content/drive/My Drive/plakshadrug/AA.csv')
df=df.iloc[:35000,:]
smiles = df['smiles'].tolist()
df.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(35000, 2)

# Preparing data for the VAE

In [528]:
# charset for smiles data

charset = set("".join(list(df.smiles))+"!E")

# padding smiles to 120 characters
char_to_int = dict( (c,i) for i,c in enumerate( charset ) )
def char_encoder( characters, maxlen=120 ):
    characters = Chem.MolToSmiles(Chem.MolFromSmiles( characters ))
    padded_char = np.zeros( ( maxlen, len( charset ) ) )
    for i, c in enumerate( characters ):
        padded_char[i, char_to_int[c] ] = 1
    return padded_char

# decoding smiles
int_to_char = dict( (i,c) for i,c in enumerate( charset ) )
def char_decoder( X ):
    X = X.argmax( axis=-1 )
    decoded_char = ''
    for i in X:
        decoded_char += int_to_char[ i ]
    return decoded_char
# onehot encoding 
one_hot_encoded_data = np.array([char_encoder(x) for x in smiles])
one_hot_encoded_data.shape


(35000, 120, 52)

In [529]:
# adding a dimension
one_hot_encoded_data = one_hot_encoded_data.reshape(35000,120,52,1)
# preparing data for training
#batch size is 32
train_dataset = (tf.data.Dataset.from_tensor_slices(one_hot_encoded_data).batch(32))

# Define VAE, encoder and decoder

In [530]:
#define VAE as a class
class VAE(tf.keras.Model):


  def __init__(self, latent_dim):
    super(VAE, self).__init__()
    self.latent_dim = latent_dim
    # defining encoder
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(120, 52, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )
    # defining decoder
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=30*13*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(30, 13, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(120, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [531]:
# Define loss fn and training step
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  x_logit = tf.cast(x_logit, tf.float64)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  logpz = tf.cast(logpz, tf.float64)
  logqz_x = tf.cast(logqz_x, tf.float64)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# train steps
model = VAE(latent_dim)
epochs = 100
print("VAE started!!")
for epoch in range(1, epochs + 1):
  for train_x in train_dataset:
    train_step(model, train_x, optimizer)
  print("epoch:", epoch, " of ", epochs, " total epochs ")
 

VAE started!!
epoch: 1  of  100  total epochs 
epoch: 2  of  100  total epochs 
epoch: 3  of  100  total epochs 
epoch: 4  of  100  total epochs 
epoch: 5  of  100  total epochs 
epoch: 6  of  100  total epochs 
epoch: 7  of  100  total epochs 
epoch: 8  of  100  total epochs 
epoch: 9  of  100  total epochs 
epoch: 10  of  100  total epochs 
epoch: 11  of  100  total epochs 
epoch: 12  of  100  total epochs 
epoch: 13  of  100  total epochs 
epoch: 14  of  100  total epochs 
epoch: 15  of  100  total epochs 
epoch: 16  of  100  total epochs 
epoch: 17  of  100  total epochs 
epoch: 18  of  100  total epochs 
epoch: 19  of  100  total epochs 
epoch: 20  of  100  total epochs 
epoch: 21  of  100  total epochs 
epoch: 22  of  100  total epochs 
epoch: 23  of  100  total epochs 
epoch: 24  of  100  total epochs 
epoch: 25  of  100  total epochs 
epoch: 26  of  100  total epochs 
epoch: 27  of  100  total epochs 
epoch: 28  of  100  total epochs 
epoch: 29  of  100  total epochs 
epoch: 30

# Decode molecules from latent space

In [532]:
# makes molecules decoded from the latent space.
def latent_space_matrix(model, lat_vars):
  
  
  norm = tfp.distributions.Normal(0, 1)
  grid_x = norm.quantile(np.linspace(0.05, 0.95, 10))
  grid_y = norm.quantile(np.linspace(0.05, 0.95, 10))
#  specifying width and height of matrix
  image = np.zeros((52, 120))

  for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
      a = lat_vars[0] if lat_vars[0] != 0 else xi
      b = lat_vars[1] if lat_vars[1] != 0 else yi
      c = np.array([[a, b]]) 
      decoded = model.sample(c)
      drug = tf.reshape(decoded[0], (120, 52))
      return drug

#convert latent space variable to one hot encoded
def convert_latent(vec_float):
  max_arg = np.argmax(vec_float)
  mol_vec = [1 if j==max_arg else 0 for j,_ in enumerate(vec_float)]
  return mol_vec

# Printing new molecules

In [534]:
# Convert and print new molecules in smile format with 120 characters

lspace=[1,3,6,9,16]
for i in range (0,5):
  mol = latent_space_matrix(model,lat_vars = [lspace[i],0])
  mol_decode = [convert_latent(i) for i in mol]
  new_drug=char_decoder(np.array(mol_decode))
  print("Drug ",i+1 ," :  ",new_drug)


Drug  1  :   C=C[C((((C)CC==NNNCNNOOOO//gg#M#g77n66##n#####77))))3#]ZCC]a2+1K6#R(.9(B6c)2..o95-2C\rB6oB@P6-(P)l@K1-=(rVcn)IeerB)KII)e
Drug  2  :   C[C@@](O))CCCCNNNO))NNONN)eet#t77777M777R#####7)M)2(@#][[K9H259PXXoX.9(]/CCC((PR]-@/33g=o9@P6-M]9lgMM-=(PPMM2577MBagF.)X
Drug  3  :   C[C@@]FOO)CCCCCNO)))))/NN)eet#t7887MM(77R8###)77MM2@@c#tgK#ns[tteXoXP#(//CZZZ#t@@@//33g=o(tFFXX]9]@@M3@@PcMK2P77M%aaFF)X
Drug  4  :   C[C@@rPOO)CC@@C(()))))/N))eO##t788(MM(77ttt#C)C7MM2@@cttgK#ns[PtSSooP33//CZZt#t@@]//#37//(tttXM99l@@3333tcMB2P@/M##gF77X
Drug  5  :   C[C@@rPOO)CC@CC(())))/tt))eOO-t78898O(OOttt#C)CHH]2O@cttgK#ns[PtSSooP33/tCZ3t#t@-]//##t//(ttttM99\ttM-@@tcBttP@/t###F77t
